# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back', 'rb'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [80]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/home/user/.local/lib/python3.10/site-packages/IPython/core/magics/pylab.py:166: UserWarning: pylab import has clobbered these variables: ['permutation', 'clf']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [81]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print (classes)

['Back', 'Knee', 'StandInit', 'Belly', 'Left', 'Frog', 'HeadBack', 'Right', 'Crouch', 'Stand', 'Sit']


In [121]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [132]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    d, t = load_pose_data(i)
    all_data += d
    all_target += t

print ('total number of data', len(all_data))

total number of data 313


In [133]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [134]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [135]:
# YOUR CODE HERE
clf.fit(all_data, all_target) 

SVC(C=100.0, gamma=0.001)

### predicting

In [136]:
clf.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])

In [137]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [138]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(all_target, clf.predict(all_data))

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.43      0.61        23
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        52
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        10
           7       1.00      0.91      0.95        11
           8       0.97      1.00      0.98        30
           9       0.89      1.00      0.94       102
          10       1.00      1.00      1.00        26

    accuracy                           0.96       313
   macro avg       0.99      0.94      0.95       313
weighted avg       0.96      0.96      0.95       313


Confusion matrix:
[[ 10   0   0   0   0   0   0   0   0  13   0]
 [  0  10   0   0   0   0   0   0   0   0   0]
 [  0   0  52   0   0   0   0   0   0   0   0]
 [  

## 4. Evaluate on the test data

In [139]:
expected = []
predicted = []
# YOUR CODE HERE

evaluate(all_target, clf.predict(all_data))

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.43      0.61        23
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        52
           3       1.00      1.00      1.00        19
           4       1.00      1.00      1.00        20
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        10
           7       1.00      0.91      0.95        11
           8       0.97      1.00      0.98        30
           9       0.89      1.00      0.94       102
          10       1.00      1.00      1.00        26

    accuracy                           0.96       313
   macro avg       0.99      0.94      0.95       313
weighted avg       0.96      0.96      0.95       313


Confusion matrix:
[[ 10   0   0   0   0   0   0   0   0  13   0]
 [  0  10   0   0   0   0   0   0   0   0   0]
 [  0   0  52   0   0   0   0   0   0   0   0]
 [  

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [140]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [141]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, "rb"))
clf2.predict(all_data), all_target

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,
         6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,
         7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9, 